Let's do the first investigation of CNVs, using the guidelines in the CNVnator paper to look for DNVs in trios.

We have 3 different window choices, so let's start with one and we can plot the results later to see how our findings change. But start by parsing the trios, so we know the different identifities.

In [10]:
import glob


data_dir = '/data/NCR_SBRB/simplex/'
trios = {}
affected = []
controls = []
peds = glob.glob(data_dir + '*trio*ped')
for ped in peds:
    trio_name = ped.split('/')[-1].split('.')[0]
    fid = open(ped, 'r')
    fam = {}
    for line in fid:
        famid, sid, fa, mo, sex, aff = line.rstrip().split('\t')
        if fa != '0':
            fam['child'] = sid
            if aff == '1':
                affected.append(trio_name)
            else:
                controls.append(trio_name)
        elif sex == '1':
            fam['father'] = sid
        else:
            fam['mother'] = sid
    trios[trio_name] = fam
    fid.close()

Now we can examine a single family and window:

In [80]:
w = 500
trio_name = '855_trio1'


import pandas as pd
import numpy as np


def get_dnvs(trio, data_dir, w):
    child = pd.read_table(data_dir + \
                          'cnvnator/%s_calls_w%d.txt' % (trio['child'], w),
                         header = False,
                         names = ['CNV_type', 'coordinates', 'CNV_size',
                                  'normalized_RD', 'e-val1', 'e-val2',
                                  'e-val3', 'e-val4', 'q0'])
    father = pd.read_table(data_dir + \
                          'cnvnator/%s_calls_w%d.txt' % (trio['father'], w),
                         header = False,
                         names = ['CNV_type', 'coordinates', 'CNV_size',
                                  'normalized_RD', 'e-val1', 'e-val2',
                                  'e-val3', 'e-val4', 'q0'])
    mother = pd.read_table(data_dir + \
                          'cnvnator/%s_calls_w%d.txt' % (trio['mother'], w),
                         header = False,
                         names = ['CNV_type', 'coordinates', 'CNV_size',
                                  'normalized_RD', 'e-val1', 'e-val2',
                                  'e-val3', 'e-val4', 'q0'])
    denovo = []
    for cnv_type in ['duplication', 'deletion']:
        cnvs = child['coordinates'][child['CNV_type'] == cnv_type]
        child_pos = pd.DataFrame([[s.split(':')[0],
                                   int(s.split(':')[1].split('-')[0]),
                                   int(s.split(':')[1].split('-')[1])]
                                  for s in cnvs],
                                columns=['chr', 'start', 'end'])
        cnvs = father['coordinates'][father['CNV_type'] == cnv_type]
        father_pos = pd.DataFrame([[s.split(':')[0],
                                   int(s.split(':')[1].split('-')[0]),
                                   int(s.split(':')[1].split('-')[1])]
                                  for s in cnvs],
                                columns=['chr', 'start', 'end'])
        cnvs = mother['coordinates'][mother['CNV_type'] == cnv_type]
        mother_pos = pd.DataFrame([[s.split(':')[0],
                                   int(s.split(':')[1].split('-')[0]),
                                   int(s.split(':')[1].split('-')[1])]
                                  for s in cnvs],
                                columns=['chr', 'start', 'end'])

        # for each child CNV, make sure there's no father or mother CNV that
        # overlap it
        for index, row in child_pos.iterrows():
            mother_start_overlap = np.sum((mother_pos['start'] < row['start']) &
                                          (mother_pos['end'] > row['start']))
            mother_end_overlap = np.sum((mother_pos['start'] < row['end']) &
                                          (mother_pos['end'] > row['end']))
            father_start_overlap = np.sum((father_pos['start'] < row['start']) &
                                          (father_pos['end'] > row['start']))
            father_end_overlap = np.sum((father_pos['start'] < row['end']) &
                                          (father_pos['end'] > row['end']))
            if mother_start_overlap + mother_end_overlap + \
               father_start_overlap + father_end_overlap == 0:
                    denovo.append(row)
    return(pd.DataFrame(denovo, index=range(len(denovo))))

Let's start witht the 500 window because the files are smaller, so they go faster:

In [85]:
dnvs_500 = {}
for trio in trios.iterkeys():
    dnvs_500[trio] = get_dnvs(trios[trio], data_dir, 500)

Although this approach might work, I just noticed that they took a different approach in the paper! Basically, for each q0-filtered CNV in the affect child, they specifically genotyped those regions in the parents, and called DNVs based on their criteria. Not sure if I'd do that for every kid, but that makes sense. I could also, as a second check, genotype those specific regions from the affected child in the unaffected child. 

OK, so according to this issue (https://github.com/abyzovlab/CNVnator/issues/86) in the CNVnator webpage, the author doesn't recommend using it for WES. But, this review paper (https://molecularcytogenetics.biomedcentral.com/articles/10.1186/s13039-017-0333-5) did use it to compare to XMHH and CONIFER. So, the main issue here is that CNVnator doesn't genotype specific regions if a window of 1000 bp is not calculated. It's hard coded in the software... weird. But when I try to calculate it for some of my subjects, it doesn't work. I get Abnormal interval errors. It's likely because it's WES, and not WGS. So, we have two issues:

* use a window that best suits our dpth of coverage
* get around the genotyping hard code that CNVnaotr has of 1000 bp

To figure out DOC, I did:

In [89]:
%%bash
module load afni


cd /data/NCR_SBRB/simplex/xhmm
for f in `ls *.DOC.sample_summary`; do tail -1 $f; done | cut -f 3 | 1d_tool.py -show_mmms -infile -

file - (len 99)
    col 0: min = 20.1100, mean = 57.3626, max = 120.0700, stdev = 13.9842


[+] Loading AFNI current-openmp ...
AFNI/current-openmp last updated  2017-11-02



The CNVnator paper (http://genome.cshlp.org/content/21/6/974.full.pdf+html) recommends ~100-bp bins for 20–30X coverage, ~500-bp bins for 4–6X coverage, and ~30-bp bins for 100X coverage. In the review paper they used  50–60bp bins for their coverage of 45–70X. So, even though my mean is around 50, which should make 50bp a good window, I have a few subjects with 20X, so I think 100bps is a safe number. Maybe once all is good I can try 50 as well just for SaGs...

But after gettting several errors, such as not being able to genotype even the window I did have (even after I created 1000bp windows in subjects that could do that!), I decided to shelf explorations with CNVnator so we can focus on other tools that might be more promising, such as XHMM, CNVkit, Conifer, ExomeDepth, ExomeCopy, and Excavator2tool.

-----

In [87]:
trios['855_trio1']

{'child': 'CLIA_400216', 'father': 'CLIA_400176', 'mother': 'CLIA_400177'}

In [24]:
child.head()

,CNV_type,coordinates,CNV_size,normalized_RD,e-val1,e-val2,e-val3,e-val4,q0
0,duplication,chr1:10051-470520,460470,3843240.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.765100
1,deletion,chr1:470521-521400,50880,530372.0,0.232258,0.000000e+00,29355.500000,0.000000e+00,1.000000
2,duplication,chr1:521401-2725920,2204520,3395790.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.112426
3,deletion,chr1:2725921-2743410,17490,97601.6,0.077662,9.070830e-167,0.477643,6.692910e-147,0.000000
4,duplication,chr1:2743411-2946210,202800,302104.0,0.000000,0.000000e+00,0.000000,0.000000e+00,0.014721


In [62]:
b = np.zeros(60000000)
print b.shape

(60000000,)
